In [1]:
import numpy as np
from scipy.stats import norm

Let $S$ be a non-dividend-paying stock that follows Geometric Brownian motion with $70\%$ volatility and time-$0$ price $S_{0} = 10$. The interest rate is $2\%$

Find the time-$0$ price of an "$\text{EY}$" contract which pays at time $1$

$$ \left(\frac{S_{0.5}+S_{1.0}}{2}- 12 \right)^+$$

Use conditional Monte Carlo, conditioning on $S_{0.5}$. 

Let:
- $T_{0} = 0.5$
- $T_{1} = 1$
- $S_{0} = 10$
- $K = 12$
- $r = 0.02$
- $R_{grow} = r$
- $\sigma = 0.70$

The conditional expected discounted payoff at time 0, given $(S_{T_{0}})$, can be expressed as:
$$\text{EY}\left(S_{T_{0}}\right) = e^{-rT_{0}} \cdot \mathbb{E}\left[Y\left(S_{T_{1}}, S_{T_{0}}\right) | S_{T_{0}}\right] 
= e^{-rT_{0}} \cdot \mathbb{E}\left[\left(\frac{S_{T_0}+S_{T_1}}{2}- 12 \right)^+ | S_{T_{0}}\right] 
$$

Note: We need to consider the additional condition when $\left(S_{T_{1}} > 2K\right)$.

Our final form for the time-$0$ price is the following:
$$\begin{align*}
C^{BS} &= C^{BS}\left(S_{t}=S_{T_{0}}, t=T_{0}, K=K, T=T_{1}, R_{grow}, r, \sigma\right) \\
CB^{BS} &= \left( S_{T_{0}} - K\right) \\
d_{2} &= \frac{\ln\left(\frac{S_{T_{0}}}{2K}\right) + \left(R_{grow} - \frac{1}{2}\sigma^{2}\right)\left(T_{1}-T_{0}\right)}{{\sigma\sqrt{T_{1}-T_{0}}}} \\
V_{0} &= \exp\left(-rT_{0}\right)\left[\mathbb{1}\{S_{T_{0}} \leq 2K \}C^{BS} + \mathbb{1}\{S_{T_{0}} > 2K\}CB^{BS} \right] \\
\end{align*}$$


In [2]:
class GBM:

    def __init__(self,S0,r,sigma=None):
        self.S0=S0
        self.r=r
        self.sigma=sigma


In [3]:
class CallOnAverage:

    def __init__(self,K,T0,T1):
        self.K = K
        self.T0 = T0
        self.T1 = T1

In [22]:
### Pricing Engine
class AnalyticEngine:

    def __init__(self):
        pass
    
    
    def BSpriceCall(self, dynamics: GBM, contract: CallOnAverage) -> float:
        ### Separate dynamics and contract class attributes to variables
        K, T0, T1 = contract.__dict__.values()
        S0, r, sigma = dynamics.__dict__.values()
        
        ### Make BS variables and call price
        timeRemaining = T1 - T0
        F = S0 * np.exp(r * timeRemaining)
        std = sigma * np.sqrt(timeRemaining)
        d1 = np.log(F / K) / std + std / 2
        d2 = d1 - std
        call_price = np.exp(-r * timeRemaining) * (F * norm.cdf(d1) - K * norm.cdf(d2))
        
        return call_price
    

class MCengine:

    def __init__(self,M,seed):
        self.M = M  #number of paths
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def randomLogreturn(self,dynamics,deltat):
        return ((dynamics.r - dynamics.sigma**2/2) * deltat 
                + dynamics.sigma*np.sqrt(deltat) * self.rng.normal(size=self.M))

    def price_CallOnAverage_GBM(self,contract,dynamics):
        #You fill this in.
        K, T0, T1 = contract.__dict__.values()
        S0, r, sigma = dynamics.__dict__.values()
        deltat1 = T0
        deltat2 = T1 - T0
        
        # Simulate S0.5
        S0_5 = S0 * np.exp(self.randomLogreturn(dynamics, deltat1))
        
        payoffs = np.zeros(self.M)
        
        # Simulate S1.0 given S0.5
        for i in range(self.M):
            if S0_5[i] < 24:
                S1_0 = S0_5[i] * np.exp(self.randomLogreturn(dynamics, deltat2)[i])
                payoffs[i] = max((S0_5[i] + S1_0)/2 - K, 0)
            else:
                payoffs[i] = S0_5[i] - 12
        
        # Discount payoffs to present value
        payoffs *= np.exp(-r * T1)
        
        # Calculate price and standard error
        price = np.mean(payoffs)
        standard_error = np.std(payoffs) / np.sqrt(self.M)

        return (price, standard_error)


In [23]:
p3dynamics = GBM(sigma=0.70,S0=10,r=0.02)

In [24]:
p3contract = CallOnAverage(K=12,T0=0.5,T1=1.0)

In [27]:
p3MC = MCengine(M=10000,seed=0)
(p3price, p3standard_error) = p3MC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print(p3price, p3standard_error)

1.5324553557623568 0.037832512015875994
